In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import noisereduce as nr
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import IPython
import os
import pyaudio
import time
import wave
from scipy.io import wavfile
import soundfile as sf
from IPython.display import Audio
import audioop
from collections import deque
import os
import urllib
import time
import math

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## To Do

* 왜 지직거리게 파일이 저장되는지 확인 필요
* pyInt16일 때 파일이 정상적으로 녹음되는 것 같은데, 그렇게 하면 float으로 받아온 데이터들이 int처리 되면서 0으로 되어서 [ParameterError: Audio buffer is not finite everywhere 에러 뜸](https://github.com/kyungyunlee/vocalMashup/issues/1)
  <br>  → fromstring으로 읽어오는 걸로 해결
* 일단 pyFloat32로 해놓은 상태인데 음질이 지직거리게 저장됨. 
   <br> → 일단 noisereduce 제외해놓은 상태 
* 너무 짧은 단위로 저장되는 중.

* noise sample 뽑을 때 이미 노이즈가 존재하는 거면 그 기준으로 loud_threshold를 잡으면 발화 도중에 소음이 있어도 silence reign에 있다고 되므로 다른 판단 필요

## int16 현재 진행 중

In [ ]:

sf.write('sample.wav',file)

In [3]:
RATE = 16000
SEC = 0.5
CHUNKSIZE = 1024
RECORD_SECONDS = 10
FORMAT = pyaudio.paInt16
CHANNELS = 1

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

#noise window
data = stream.read(10000,exception_on_overflow=False)
noise_sample = np.fromstring(data, dtype=np.int16)
print("Noise Sample")

loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def save_file(data,p):
    foldername = 'noise_reduction_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
    createFolder(foldername)
    filename = foldername + 'nr_output_'+str(int(time.strftime('%H%M%S')))
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)  
    wf.writeframes(data)
    wf.close()
    return filename + '.wav'
#    return sf.write(f'{filename}'+'.wav')
try:
    while(True):
        # Read chunk and load it into numpy array.
        data = stream.read(CHUNKSIZE,exception_on_overflow=False)
        current_window = np.fromstring(data, dtype=np.int16)
        print(current_window)

        if(audio_buffer==[]):
            audio_buffer = current_window
        else:
            if(np.mean(np.abs(current_window))<loud_threshold):
                print("Inside silence reign")       
                if(near<100):
                    audio_buffer = np.concatenate((audio_buffer,current_window))
                    near += 1
                else:
                    predictSound(np.array(audio_buffer))
                    audio_buffer = []
                    near
            else:
                print("Inside loud reign")
                near = 0
                audio_buffer = np.concatenate((audio_buffer,current_window))
except:
    file = audio_buffer
    filename = save_file(file,p)  ##
        
# close stream
stream.stop_stream()
stream.close()
p.terminate()

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Noise Sample
Loud threshold 6351.394
[ 434  573  564 ... -907 -825 -615]
[-390 -105  181 ...  412  649  816]
Inside silence reign
[  872   857   840 ... -1263 -1075  -903]
Inside silence reign
[-787 -668 -513 ...  227  113  144]
Inside silence reign
[ -24 -177 -280 ...   97  166  234]
Inside silence reign
[ 300  380  358 ... -913 -910 -915]
Inside silence reign
[-1006  -977  -989 ...   465   508   591]
Inside silence reign
[586 559 558 ... 764 906 966]
Inside silence reign
[ 884  864 1046 ...  219  372  589]
Inside silence reign
[815 880 849 ... 144 285 420]
Inside silence reign
[ 572  691  733 ... 2437 2276 2221]
Inside silence reign
[2035 1782 1614 ... -327 -645 -853]
Inside silence reign
[ -924  -995 -1122 ...   169    40   -22]
Inside silence reign
[ -142  -249  -326 ... -1069 -1246 -1411]
Inside silence reign
[-1492 -1503 -1570 ... -1022  -869  -509]
Inside silence reign
[ -635  -413  -135 ... -1452 -1842 -2260]
Inside silence reign
[-2364 -2428 -2311 ...  -243  -770  -619]
Inside

## 침묵구간 큐에 저장해두고 하기

## threshold 바꿔서 해보기

In [4]:
RATE = 16000
SEC = 0.5
CHUNKSIZE = int(RATE*SEC) # fixed chunk size
#CHUNKSIZE = 1024
RECORD_SECONDS = 10
FORMAT = pyaudio.paInt16
CHANNELS = 1

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

In [5]:
#noise window
data = stream.read(CHUNKSIZE*10,exception_on_overflow=False)
noise_sample = np.fromstring(data, dtype=np.int16)
print("Noise Sample")
#loud_threshold = np.mean(np.abs(noise_sample)) * 3
loud_threshold = math.sqrt(abs(audioop.avg(noise_sample,4)))
print("Loud threshold", loud_threshold)

stream.stop_stream()
stream.close()
p.terminate()

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until


Noise Sample
Loud threshold 221.66190471075538


In [6]:
import IPython
IPython.display.Audio(data=noise_sample, rate=16000)

## scipy.io.wavfile.write ㅣlibrosa
## format 체크하기

## Float32

In [2]:
CHUNKSIZE = 1024 # fixed chunk size
RATE = 16000
RECORD_SECONDS = 10
FORMAT = pyaudio.paFloat32
CHANNELS = 1

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

#noise window
data = stream.read(10000,exception_on_overflow=False)
noise_sample = np.fromstring(data, dtype=np.float32)
print("Noise Sample")

loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def save_file(data,p):
    foldername = 'noise_reduction_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
    createFolder(foldername)
    filename = foldername + 'nr_output_'+str(int(time.strftime('%H%M%S')))
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)  
    wf.writeframes(data)
    wf.close()
    return filename + '.wav'
try:
    while(True):
        # Read chunk and load it into numpy array.
        data = stream.read(CHUNKSIZE,exception_on_overflow=False)
        current_window = np.fromstring(data, dtype=np.float32)
        current_window = nr.reduce_noise(audio_clip=current_window, noise_clip=noise_sample, verbose=False)

        print(current_window)

        if(audio_buffer==[]):
            audio_buffer = current_window
        else:
            if(np.mean(np.abs(current_window))<loud_threshold):
                print("Inside silence reign")       
                if(near<100):
                    audio_buffer = np.concatenate((audio_buffer,current_window))
                    #print('audio_buffer:',audio_buffer)
                    near += 1
                else:
                    audio_buffer = []
                    near
            else:
                print("Inside loud reign")
                near
                audio_buffer = np.concatenate((audio_buffer,current_window))
except:
    file = audio_buffer
    #filename = save_file(file, p)
        
# close stream
stream.stop_stream()
stream.close()
p.terminate()

<ipython-input-2-acfa5244aa58>:13: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  noise_sample = np.fromstring(data, dtype=np.float32)
<ipython-input-2-acfa5244aa58>:44: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  current_window = np.fromstring(data, dtype=np.float32)
/Users/jaeyoungshin/opt/anaconda3/lib/python3.8/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):
/Users/jaeyoungshin/opt/anaconda3/lib/python3.8/site-packages/librosa/util/utils.py:343: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `

Noise Sample
Loud threshold 0.027668250259011984
[ 0.00097723  0.0013278   0.00141332 ... -0.00752809 -0.00660836
 -0.00610796]
[-0.00607743 -0.00549543 -0.00499633 ...  0.01235961  0.01167733
  0.01175239]
Inside silence reign
[0.01043487 0.01030102 0.01023563 ... 0.00240315 0.00224914 0.00216512]
Inside silence reign
[-0.00233949 -0.00229616 -0.00260791 ... -0.00126675 -0.00122353
 -0.00128306]
Inside silence reign
[-0.00187074 -0.00185035 -0.00188001 ... -0.00012945 -0.00026696
 -0.00033776]
Inside silence reign
[-3.0619730e-04 -3.5063035e-04 -4.0688450e-04 ... -4.6219237e-04
 -1.5066509e-04 -3.0568224e-06]
Inside silence reign
[0.00027365 0.00062625 0.00057931 ... 0.00090455 0.00089616 0.00084113]
Inside silence reign
[0.0006202  0.00067133 0.00068637 ... 0.00046665 0.00041234 0.00040136]
Inside silence reign
[-1.1804594e-05  7.2328530e-06  2.3459126e-05 ... -1.5089892e-04
 -5.2549134e-05  3.4131557e-05]
Inside silence reign
[0.00016451 0.00022956 0.00028153 ... 0.00047908 0.000474

[ 0.00580458  0.00575845  0.00567895 ... -0.00071006 -0.00075126
 -0.00078738]
Inside silence reign
[-9.6795533e-04 -1.0512425e-03 -1.1534096e-03 ...  9.3261166e-05
  8.3823303e-05  1.4366642e-04]
Inside silence reign
[3.2959584e-04 1.9229636e-04 3.8059599e-05 ... 1.8784842e-03 1.7743363e-03
 1.7015500e-03]
Inside silence reign
[ 0.0017513   0.00166809  0.00161047 ... -0.00044605 -0.00042222
 -0.0004049 ]
Inside silence reign
[-1.8631945e-04 -8.3779909e-05  6.0571219e-06 ... -6.2641053e-04
 -6.3376926e-04 -7.1606151e-04]
Inside silence reign
[-0.0015332  -0.00154013 -0.00153674 ... -0.00097957 -0.00094113
 -0.00092939]
Inside silence reign
[-0.00032466 -0.00037811 -0.00035485 ... -0.00123897 -0.00119685
 -0.00107979]
Inside silence reign
[-0.00049992 -0.00040845 -0.00036835 ... -0.00164096 -0.00155353
 -0.00145871]
Inside silence reign
[-0.00137199 -0.00133581 -0.00130438 ... -0.00117196 -0.00113005
 -0.00110453]
Inside silence reign
[-0.0009983  -0.00102541 -0.00097132 ... -0.00151393

## 저장 한번에

In [8]:
RATE = 16000
CHUNKSIZE = 1024 # fixed chunk size
RECORD_SECONDS = 10
FORMAT = pyaudio.paFloat32
CHANNELS = 1

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

#noise window
data = stream.read(10000,exception_on_overflow=False)
noise_sample = np.fromstring(data, dtype=np.float32)
print("Noise Sample")
loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def save_file(data,p):
    foldername = 'noise_reduction_audio/'+str(time.strftime('%Y-%m-%d'))+'/'
    createFolder(foldername)
    filename = foldername + 'nr_output_'+str(int(time.strftime('%H%M%S')))
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)  
    wf.writeframes(data)
    wf.close()
    return filename + '.wav'
try:
    while(True):
        # Read chunk and load it into numpy array.
        data = stream.read(CHUNKSIZE,exception_on_overflow=False)
        current_window = np.fromstring(data, dtype=np.float32)
        current_window = nr.reduce_noise(audio_clip=current_window, noise_clip=noise_sample, verbose=False)

        print(current_window)

        if(audio_buffer==[]):
            audio_buffer = current_window
        else:
            if(np.mean(np.abs(current_window))<loud_threshold):
                print("Inside silence reign")       
                if(near<100):
                    audio_buffer = np.concatenate((audio_buffer,current_window))
                    #print('audio_buffer:',audio_buffer)
                    near += 1
                else:
                    audio_buffer = []
                    near
            else:
                print("Inside loud reign")
                near = 0
                audio_buffer = np.concatenate((audio_buffer,current_window))
except:
    file = audio_buffer
    filename = save_file(file, p)
        
# close stream
stream.stop_stream()
stream.close()
p.terminate()

/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  del sys.path[0]
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1536
  n_fft, y.shape[-1]


Noise Sample
Loud threshold 0.3550552949309349


/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1024
  n_fft, y.shape[-1]
/Users/jaeyoungshin/opt/anaconda3/envs/python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:48: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


[-0.01569308 -0.0058035   0.00163456 ...  0.03746778  0.04377558
  0.0543793 ]
[0.05421807 0.05555889 0.05701808 ... 0.02627074 0.02914031 0.03746069]
Inside silence reign
[ 0.03487644  0.03516832  0.03314547 ...  0.00608837  0.00143414
 -0.01021003]
Inside silence reign
[-0.02026054 -0.02296615 -0.02543178 ... -0.03794994 -0.03383513
 -0.02616695]
Inside silence reign
[-0.02292479 -0.0167952  -0.0085356  ... -0.003556   -0.00147686
 -0.00034111]
Inside silence reign
[-0.00186562 -0.0007026  -0.00066391 ...  0.02486549  0.02511618
  0.02706402]
Inside silence reign
[ 0.02559369  0.02553442  0.0259117  ... -0.04420379 -0.03993763
 -0.03809933]
Inside silence reign
[-0.03452718 -0.03131206 -0.02670774 ...  0.01851287  0.01667989
  0.02036082]
Inside silence reign
[0.01920384 0.01389373 0.01690193 ... 0.00330339 0.0054895  0.0068718 ]
Inside silence reign
[ 0.00335191  0.00197031  0.00332133 ... -0.01440539 -0.0018254
 -0.0057464 ]
Inside silence reign
[-0.00925889  0.0065752  -0.00145746

[-0.04364447 -0.03807867 -0.03048347 ...  0.02695733  0.0295268
  0.03611338]
Inside silence reign
[0.03861722 0.04391315 0.05050158 ... 0.01637256 0.01414783 0.00883716]
Inside silence reign
[-0.00065374 -0.01146922 -0.01189305 ...  0.00927829  0.00396692
  0.01121592]
Inside silence reign
[ 0.01943602  0.01644279  0.01212498 ...  0.01255481  0.00701253
 -0.00877453]
Inside silence reign
[-0.01792274 -0.02507967 -0.03601102 ...  0.01086558  0.0191155
  0.02628059]
Inside silence reign
[0.02350325 0.02104464 0.02402298 ... 0.04907973 0.04370337 0.04496313]
Inside silence reign
[ 0.03892993  0.03659793  0.03246722 ... -0.00967772 -0.0063585
 -0.00633673]
Inside silence reign
[-0.01391831 -0.01098504 -0.00334994 ... -0.00300655  0.00020057
  0.00579064]
Inside silence reign
[ 0.00598951  0.00739119  0.01628715 ... -0.00835188 -0.01061478
 -0.01188378]
Inside silence reign
[-0.01604292 -0.01599854 -0.01932761 ... -0.020208   -0.01996914
 -0.02055227]
Inside silence reign
[-0.001821   -0.0

[-0.04882234 -0.05606776 -0.04587303 ...  0.05789737  0.05835479
  0.0514435 ]
Inside silence reign
[ 0.04932038  0.06014393  0.05738307 ...  0.00703502 -0.00525518
 -0.01508786]
[-0.01065717 -0.02183814 -0.03575064 ... -0.03234035 -0.04038088
 -0.04297708]
Inside silence reign
[-0.03403833 -0.02998348 -0.0301444  ... -0.02348472 -0.01760541
 -0.00488487]
[-0.01764484  0.00010884  0.0216882  ... -0.03733648 -0.03426953
 -0.03437288]
Inside silence reign
[-0.03090261 -0.02176307 -0.01957378 ... -0.09900768 -0.09841332
 -0.08194003]
[-0.05748875 -0.03531924 -0.03610277 ...  0.03724855  0.04122642
  0.03306251]
Inside silence reign
[ 0.02678447  0.02455706  0.01880036 ... -0.07283004 -0.07859097
 -0.0875561 ]
[-0.08869298 -0.09170103 -0.09478312 ... -0.00958955 -0.00768014
 -0.02077783]
Inside silence reign
[-0.01895071 -0.01092381 -0.0155933  ... -0.05128202 -0.04791986
 -0.05060121]
[-0.0471632  -0.04231389 -0.03713931 ... -0.07052971 -0.07620896
 -0.08428357]
Inside silence reign
[-0.1

[-0.00668509 -0.00219631  0.00050523 ... -0.0109888  -0.00976846
 -0.01118019]
Inside silence reign
[-0.01335957 -0.01300904 -0.01194814 ... -0.00337648  0.00091062
  0.00528883]
[ 0.01224382  0.01320251  0.01537799 ... -0.00439518 -0.00344525
 -0.00407814]
Inside silence reign
[-0.00390644 -0.00647942 -0.00722274 ... -0.02754024 -0.02452426
 -0.02168607]
[-0.0228704  -0.02398026 -0.02512091 ...  0.00662335  0.00354138
  0.00031703]
Inside silence reign
[-0.00320237 -0.00692897 -0.01076212 ...  0.02092735  0.02258662
  0.02618043]
[0.02829228 0.02967308 0.03115984 ... 0.02052036 0.02072728 0.02255993]
Inside silence reign
[ 0.02580992  0.02744467  0.02626917 ... -0.01365857 -0.01022341
 -0.01063499]
[-0.01484007 -0.01541681 -0.0170507  ...  0.01136861  0.00763455
  0.00319926]
Inside silence reign
[ 0.0004799  -0.00176212 -0.00358342 ...  0.02043337  0.01810499
  0.01863089]
[ 0.02321822  0.02442329  0.02797238 ... -0.00109793 -0.00444157
 -0.00528908]
Inside silence reign
[-0.00807165